
 ETL - Média de investimento por cliente (Anúncios Meta)

 **Contexto do projeto**
 Você recebeu dados (CSV) contendo registros de compras/investimentos de clientes em serviços de anúncios no Meta.
 Seu objetivo é:

 1) Gerar dados sintéticos e salvar em CSV  
 2) Ler o CSV e validar os registros  
 3) Calcular a média de investimento (spend) por cliente  
 4) Exportar o resultado em JSON

 **Formato esperado do CSV**
 - client_id, spend

 **Formato esperado do JSON**
 ```json
 {
   "C001": 1234.56,
   "C002": 987.65
 }



In [2]:
!pip install numpy pandas


  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----------- ---------------------------- 3.7/12.6 MB 19.8 MB/s eta 0:00:01
   -------------------- ------------------- 6.6/12.6 MB 16.1 MB/s eta 0:00:01
   ---------------------------------- ----- 10.7/12.6 MB 17.7 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 16.8 MB/s  0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------- ----------------------- 4.2/9.9 MB 20.9 MB/s eta 0:00:01
   ------------------------------- -------- 7.9/9.9 MB 18.7 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 17.1 MB/s  0:00:00
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)

   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ----------------------

In [3]:
from __future__ import annotations

import json
import math
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd


@dataclass(frozen=True)
class Config:
    csv_path: Path = Path("meta_clientes.csv")
    json_path: Path = Path("resultado_media_por_cliente.json")
    n_registros: int = 200
    clientes: Tuple[str, ...] = ("C001", "C002", "C003", "C004", "C005", "C006")
    seed: int = 42


CFG = Config()

 1) Geração de dados sintéticos (CSV)
 Vamos simular compras/investimentos em mídia (spend) por cliente.
 - Alguns clientes gastam menos (pequenas contas)
 - Outros gastam mais (contas maiores)
 - Também incluiremos poucos registros “ruins” de propósito (para testar validação)

In [5]:
import csv
import math
from pathlib import Path
import numpy as np
import pandas as pd

def gerar_csv_sintetico_meta(csv_path: Path, n_registros: int = 200, seed: int = 42) -> None:
    rng = np.random.default_rng(seed)

    clientes = ["C001", "C002", "C003", "C004", "C005", "C006"]
    faixas = {
        "C001": (300, 1500),
        "C002": (500, 2500),
        "C003": (800, 6000),
        "C004": (200, 1200),
        "C005": (1500, 12000),
        "C006": (1000, 9000),
    }

    linhas = []
    for i in range(n_registros):
        cid = rng.choice(clientes)
        lo, hi = faixas[cid]
        valor = rng.lognormal(mean=math.log((lo + hi) / 2), sigma=0.35)
        valor = float(np.clip(valor, lo, hi))
        linhas.append([cid, f"{valor:.2f}"])  # armazenar como TEXTO

    # Inserir inválidos para teste (também como TEXTO)
    if n_registros >= 30:
        linhas[5][1]  = ""       # spend vazio
        linhas[12][0] = ""       # client_id vazio
        linhas[18][1] = "abc"    # spend não numérico
        linhas[25][1] = "-50"    # spend negativo

    csv_path.parent.mkdir(parents=True, exist_ok=True)
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["client_id", "spend"])
        w.writerows(linhas)

# uso
CSV_PATH = Path("meta_clientes.csv")
gerar_csv_sintetico_meta(CSV_PATH, n_registros=200, seed=42)

df_raw = pd.read_csv(CSV_PATH, encoding="utf-8")
df_raw.head(10)


,client_id,spend
0,C001,625.41
1,C005,8777.58
2,C001,454.65
3,C005,4279.26
4,C005,6042.74
5,C005,NaN
6,C006,6802.07
7,C003,4463.81
8,C005,10014.93
9,C004,824.44


 2) Extract (leitura) + checagens iniciais
 Vamos carregar o CSV e conferir:
 - colunas obrigatórias
 - tamanho
 - amostra dos dados

In [6]:
def ler_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {path}")
    df = pd.read_csv(path, encoding="utf-8")
    return df


df_raw = ler_csv(CFG.csv_path)

colunas_obrigatorias = {"client_id", "spend"}
faltando = colunas_obrigatorias - set(df_raw.columns)

print("Linhas lidas:", len(df_raw))
print("Colunas:", list(df_raw.columns))
if faltando:
    raise ValueError(f"CSV inválido. Faltam colunas: {sorted(faltando)}")

df_raw.sample(8, random_state=CFG.seed)

Linhas lidas: 200
Colunas: ['client_id', 'spend']


,client_id,spend
95,C004,768.76
15,C003,3220.99
30,C006,6783.13
158,C002,707.45
128,C005,11257.65
115,C001,586.07
69,C005,4868.45
170,C002,1376.60


 3) Transform (limpeza + agregação)
 **Regras de validação**
 - `client_id` deve existir e não ser vazio
 - `spend` deve ser numérico e maior que zero

 Em seguida:
 - agrupar por `client_id`
 - calcular a média de `spend`
 - arredondar para 2 casas

In [7]:
def limpar_e_validar(df: pd.DataFrame) -> tuple[pd.DataFrame, Dict[str, int]]:
    df = df.copy()

    # Normalizar client_id (ex.: remover espaços)
    df["client_id"] = df["client_id"].astype(str).str.strip()

    # Converter spend para numérico (forçando inválidos para NaN)
    df["spend"] = pd.to_numeric(df["spend"], errors="coerce")

    # Máscaras de validade
    m_client_ok = df["client_id"].notna() & (df["client_id"] != "") & (df["client_id"] != "nan")
    m_spend_ok = df["spend"].notna() & (df["spend"] > 0)

    df_validos = df[m_client_ok & m_spend_ok].copy()
    df_invalidos = df[~(m_client_ok & m_spend_ok)].copy()

    auditoria = {
        "total_linhas": len(df),
        "validas": len(df_validos),
        "invalidas": len(df_invalidos),
        "invalidas_client_id": int((~m_client_ok).sum()),
        "invalidas_spend": int((~m_spend_ok).sum()),
    }

    return df_validos, auditoria


def calcular_media_por_cliente(df_validos: pd.DataFrame) -> Dict[str, float]:
    medias = (
        df_validos
        .groupby("client_id", as_index=True)["spend"]
        .mean()
        .round(2)
        .sort_index()
    )
    # Converter para dict padrão {client_id: media}
    return {cid: float(valor) for cid, valor in medias.items()}


df_validos, auditoria = limpar_e_validar(df_raw)
resultado = calcular_media_por_cliente(df_validos)

print("Auditoria:", auditoria)
print("\nMédia de investimento (spend) por cliente:")
for k, v in resultado.items():
    print(f"- {k}: {v}")

Auditoria: {'total_linhas': 200, 'validas': 196, 'invalidas': 4, 'invalidas_client_id': 1, 'invalidas_spend': 3}

Média de investimento (spend) por cliente:
- C001: 875.64
- C002: 1489.34
- C003: 3557.5
- C004: 680.24
- C005: 6835.93
- C006: 5581.13


#  4) Load (exportar para JSON)
# Vamos persistir o dicionário de médias em um arquivo `.json`.

In [8]:
def salvar_json(obj: Dict[str, float], path: Path) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)


salvar_json(resultado, CFG.json_path)
print(f"JSON gerado: {CFG.json_path.resolve()}")

JSON gerado: C:\DATA\salva_3\data_D\LARC\Estudo\Phyton\resultado_media_por_cliente.json
